## Run "MuMo-CoCo" Excel CO<sub>2</sub> transportation cost model

https://docs.xlwings.org/en/stable/quickstart.html

### import modules

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw

### constants

In [2]:
# molar mass (kg/kmol)
molar_mass_H2_kg_per_kmol = 2.016
molar_mass_CO2_kg_per_kmol = 44.009

# stoichiometric ratio (mol/mol)
stoic_mol_H2_per_mol_CO2 = 1.0

# kg per tonne
kg_per_tonne = 1000.0

# tonnes per kt
tonne_per_kt = 1000.0

# days per year
# implicit assumption: 100% capacity factor
day_per_yr = 365.0

### user input

In [3]:
# average inflation rate 
# used in the CO2 transportation model
avg_infl = 0.023

# output dollar year
output_dollar_year = 2022

In [4]:
# hydrogen refueling station capacity (kg H2/day)
stn_cap_kg_H2_per_day = np.concatenate((
    np.arange(50, 1000, 50),
    np.arange(1000, 2501, 100)
))

stn_cap_kg_H2_per_day

array([  50,  100,  150,  200,  250,  300,  350,  400,  450,  500,  550,
        600,  650,  700,  750,  800,  850,  900,  950, 1000, 1100, 1200,
       1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300,
       2400, 2500])

In [5]:
# transport distance (mile)
distance_mi = np.concatenate((
    np.arange(10, 100, 10),
    np.arange(100, 1001, 100)
))

distance_mi

array([  10,   20,   30,   40,   50,   60,   70,   80,   90,  100,  200,
        300,  400,  500,  600,  700,  800,  900, 1000])

### calculations

In [6]:
# calculate CO2 flowrate to recycle (ktCO2/yr)
# assuming perfect reactions, no losses, etc.
size_kt_co2_per_yr = stn_cap_kg_H2_per_day * day_per_yr * \
    molar_mass_CO2_kg_per_kmol / molar_mass_H2_kg_per_kmol / \
    stoic_mol_H2_per_mol_CO2 / kg_per_tonne / tonne_per_kt

size_kt_co2_per_yr

array([ 0.39839497,  0.79678993,  1.1951849 ,  1.59357986,  1.99197483,
        2.39036979,  2.78876476,  3.18715972,  3.58555469,  3.98394965,
        4.38234462,  4.78073958,  5.17913455,  5.57752951,  5.97592448,
        6.37431944,  6.77271441,  7.17110938,  7.56950434,  7.96789931,
        8.76468924,  9.56147917, 10.3582691 , 11.15505903, 11.95184896,
       12.74863889, 13.54542882, 14.34221875, 15.13900868, 15.93579861,
       16.73258854, 17.52937847, 18.3261684 , 19.12295833, 19.91974826])

### run model and write output

In [7]:
# create lists
list_input_values = []
list_gross_costs = []

In [8]:
# open workbook 
wb = xw.Book('MuMo-CoCo_v0_S2A.xlsm')

# instantiate "user interface" sheet
ui = wb.sheets['User Interface']

In [9]:
# update number format
ui['F6:F9'].number_format = 'general'

In [10]:
# calculate conversion (multiplier) from input dollar year to 
# output dollar year
dollar_year_multiplier = (
    1 + avg_infl)**(output_dollar_year - ui['start_year'].value)

In [11]:
# read in input and result names
input_names = ui['A4:A30'].value
gross_cost_names = ui['D6:D9'].value

# update result names
gross_cost_names = [
    name + ' ($/t-CO2 gross)' for name in gross_cost_names]

In [12]:
input_names

['Size (kt-CO2/y)',
 'Distance (mi)',
 'Geographical Scale',
 'Location',
 'Transport Mode',
 'Container Type',
 'Rail Transit Time Method',
 'Rail Transit Time (h)',
 'Truck Long Haul Solution',
 'Electricity Source',
 'Grid Decarbonization Pathway',
 'Grid Net Zero Year',
 'GHG accounting',
 'Fuel Economy Pathway',
 "Negotiated Rail 'Base Rate' Reduction",
 'Water content (ppm-mol)',
 'CO2 Pressure (bar)',
 'Operating Period (y)',
 'Construction Period (y)',
 'Construction Start Year',
 'FINEX Parameters',
 'IRR',
 'Depreciation Period (y)',
 'Capitalization (% equity)',
 'Cost of equity (IRROE)',
 'Cost of debt (interest rate)',
 'Project Contingency Factor']

In [13]:
gross_cost_names

['Total ($/t-CO2 gross)',
 'CAPEX ($/t-CO2 gross)',
 'OPEX ($/t-CO2 gross)',
 'FINEX ($/t-CO2 gross)']

In [14]:
# run model for user-defined flowrates and distances
for ui['size_kt_co2_per_yr'].value in size_kt_co2_per_yr:
    for ui['distance_mi'].value in distance_mi:
        
        # print message
        print('Running: {:0.1f} ktCO2/yr, {:0.0f} miles...'.format(
            ui['size_kt_co2_per_yr'].value, 
            ui['distance_mi'].value))
        
        # convert cost results to output dollar year
        gross_costs = \
            ui['F6:F9'].options(np.array).value * dollar_year_multiplier
        
        # append results
        list_input_values.append(ui['B4:B30'].value)
        list_gross_costs.append(gross_costs)
        
print('Done.')

Running: 0.4 ktCO2/yr, 10 miles...
Running: 0.4 ktCO2/yr, 20 miles...
Running: 0.4 ktCO2/yr, 30 miles...
Running: 0.4 ktCO2/yr, 40 miles...
Running: 0.4 ktCO2/yr, 50 miles...
Running: 0.4 ktCO2/yr, 60 miles...
Running: 0.4 ktCO2/yr, 70 miles...
Running: 0.4 ktCO2/yr, 80 miles...
Running: 0.4 ktCO2/yr, 90 miles...
Running: 0.4 ktCO2/yr, 100 miles...
Running: 0.4 ktCO2/yr, 200 miles...
Running: 0.4 ktCO2/yr, 300 miles...
Running: 0.4 ktCO2/yr, 400 miles...
Running: 0.4 ktCO2/yr, 500 miles...
Running: 0.4 ktCO2/yr, 600 miles...
Running: 0.4 ktCO2/yr, 700 miles...
Running: 0.4 ktCO2/yr, 800 miles...
Running: 0.4 ktCO2/yr, 900 miles...
Running: 0.4 ktCO2/yr, 1000 miles...
Running: 0.8 ktCO2/yr, 10 miles...
Running: 0.8 ktCO2/yr, 20 miles...
Running: 0.8 ktCO2/yr, 30 miles...
Running: 0.8 ktCO2/yr, 40 miles...
Running: 0.8 ktCO2/yr, 50 miles...
Running: 0.8 ktCO2/yr, 60 miles...
Running: 0.8 ktCO2/yr, 70 miles...
Running: 0.8 ktCO2/yr, 80 miles...
Running: 0.8 ktCO2/yr, 90 miles...
Running: 

Running: 12.0 ktCO2/yr, 60 miles...
Running: 12.0 ktCO2/yr, 70 miles...
Running: 12.0 ktCO2/yr, 80 miles...
Running: 12.0 ktCO2/yr, 90 miles...
Running: 12.0 ktCO2/yr, 100 miles...
Running: 12.0 ktCO2/yr, 200 miles...
Running: 12.0 ktCO2/yr, 300 miles...
Running: 12.0 ktCO2/yr, 400 miles...
Running: 12.0 ktCO2/yr, 500 miles...
Running: 12.0 ktCO2/yr, 600 miles...
Running: 12.0 ktCO2/yr, 700 miles...
Running: 12.0 ktCO2/yr, 800 miles...
Running: 12.0 ktCO2/yr, 900 miles...
Running: 12.0 ktCO2/yr, 1000 miles...
Running: 12.7 ktCO2/yr, 10 miles...
Running: 12.7 ktCO2/yr, 20 miles...
Running: 12.7 ktCO2/yr, 30 miles...
Running: 12.7 ktCO2/yr, 40 miles...
Running: 12.7 ktCO2/yr, 50 miles...
Running: 12.7 ktCO2/yr, 60 miles...
Running: 12.7 ktCO2/yr, 70 miles...
Running: 12.7 ktCO2/yr, 80 miles...
Running: 12.7 ktCO2/yr, 90 miles...
Running: 12.7 ktCO2/yr, 100 miles...
Running: 12.7 ktCO2/yr, 200 miles...
Running: 12.7 ktCO2/yr, 300 miles...
Running: 12.7 ktCO2/yr, 400 miles...
Running: 12.7

In [15]:
# create dataframe for inputs
df_inputs = pd.DataFrame(
    list_input_values, 
    columns=input_names
)

# create dataframe for results
df_results = pd.DataFrame(
    list_gross_costs, 
    columns=gross_cost_names
)

# concatenate dataframes
df = pd.concat([df_inputs, df_results], axis='columns')

# add output dollar year column
df['Output Dollar Year (User Input)'] = output_dollar_year
df = df[[df.columns[-1]] + df.columns[:-1].tolist()]

# save dataframe
df.to_csv('../liq_co2_trucking_costs.csv', index=False)

In [16]:
df.columns.tolist()

['Output Dollar Year (User Input)',
 'Size (kt-CO2/y)',
 'Distance (mi)',
 'Geographical Scale',
 'Location',
 'Transport Mode',
 'Container Type',
 'Rail Transit Time Method',
 'Rail Transit Time (h)',
 'Truck Long Haul Solution',
 'Electricity Source',
 'Grid Decarbonization Pathway',
 'Grid Net Zero Year',
 'GHG accounting',
 'Fuel Economy Pathway',
 "Negotiated Rail 'Base Rate' Reduction",
 'Water content (ppm-mol)',
 'CO2 Pressure (bar)',
 'Operating Period (y)',
 'Construction Period (y)',
 'Construction Start Year',
 'FINEX Parameters',
 'IRR',
 'Depreciation Period (y)',
 'Capitalization (% equity)',
 'Cost of equity (IRROE)',
 'Cost of debt (interest rate)',
 'Project Contingency Factor',
 'Total ($/t-CO2 gross)',
 'CAPEX ($/t-CO2 gross)',
 'OPEX ($/t-CO2 gross)',
 'FINEX ($/t-CO2 gross)']